In [9]:
import nltk
from nltk.wsd import lesk
from pywsd.similarity import max_similarity,similarity_by_path
from pywsd.lesk import simple_lesk, adapted_lesk, cosine_lesk,original_lesk

from nltk.tokenize import PunktSentenceTokenizer,sent_tokenize, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet as wn
import string

import support.scrape as scrape


stop_words = set(stopwords.words('english')) 

def simpleFilter(sentence):
    filtered_sent = []
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(sentence)

    for w in words:
        if w not in stop_words:
            filtered_sent.append(lemmatizer.lemmatize(w))

    return filtered_sent

def listToString(inputSentList):
    return ' '.join(inputSentList)

def viewAllDefinitions(wordStr):
    resList = []
    for syn in wn.synsets(wordStr):
        resList.append(syn.definition())
    return resList

def topRelevSent(targetWord):
    return

def simpleFilter(sentence):
    filtered_sent = []
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(sentence)

    for w in words:
        if w not in stop_words:
            filtered_sent.append(lemmatizer.lemmatize(w))
    return listToString(filtered_sent)

def simpleFilterList(sentList):
    resList = []
    for sent in sentList:
        sent = sent.translate(str.maketrans('', '', string.punctuation))
        resList.append(simpleFilter(sent))
    return resList





sent = 'His card was the 8 of hearts'
ambiguous = 'heart'

sent = simpleFilter(sent)


print(sent)
cosine_lesk(sent,ambiguous).definition()



# original_lesk seems to be better

His card 8 heart


'a playing card in the major suit that has one or more red hearts on it'

In [10]:

        
viewAllDefinitions('bank')     


['sloping land (especially the slope beside a body of water)',
 'a financial institution that accepts deposits and channels the money into lending activities',
 'a long ridge or pile',
 'an arrangement of similar objects in a row or in tiers',
 'a supply or stock held in reserve for future use (especially in emergencies)',
 'the funds held by a gambling house or the dealer in some gambling games',
 'a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force',
 'a container (usually with a slot in the top) for keeping money at home',
 'a building in which the business of banking transacted',
 'a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)',
 'tip laterally',
 'enclose with a bank',
 'do business with a bank or keep an account at a bank',
 'act as the banker in a game or in gambling',
 'be in the banking business',
 'put into a bank account',
 'cover with ashes so to co

In [11]:
# nltk
sent = 'I went to the bank to deposit my money'
ambiguous = 'bank'
original_lesk(sent,ambiguous).definition()

'a financial institution that accepts deposits and channels the money into lending activities'

In [12]:
original_lesk('my cat likes to eat mice', 'cat').definition()

'feline mammal usually having thick soft fur and no ability to roar: domestic cats; wildcats'

In [13]:
#sent = 'She is mean to me'
#sent = 'She mean me'
sent = 'What do you mean'

original_lesk(sent, 'mean').definition()
#simple_lesk(sent, 'mean').definition()

'mean or intend to express or convey'

In [14]:
# 'run'  'advice' , 'bat', 'project'  'fine'  'consoil'  'roll'  ''
sent = 'The rebel seized the opportunity to rebel'
adapted_lesk(sent, 'rebel').definition()

'break with established customs'

In [25]:
def seperateByDef(targetWord):
    # Returns a dictionary sorted by defintion
    sentList = scrape.scrape(targetWord)
    
    dictDef = {}
    for i,sent in enumerate(sentList):
        
        #defineSent = cosine_lesk(sent,targetWord).definition()
        defineSent = adapted_lesk(sent,targetWord).definition()        
        
        if defineSent not in dictDef:
            dictDef[defineSent] = [sent]
        else:
            dictDef[defineSent].append(sent)
    return dictDef


dictDef = seperateByDef('bank')


for key in dictDef.keys():
    print('-------------------------------------------------------')
    print('Def: ', key)
    for value in dictDef[key]:
        print('        ', value)


------------------------
Def:  have confidence or faith in
         A bank of clouds was building to the northeast.
         Mr. Catlin at the bank says he's as honest as they come.
         Seating herself on a low bank, she studied the souls.
         Within an hour, they were riding side-by-side down the south bank of the creek, searching for the blocked area.
         ATMs replaced human bank tellers, so they are called "Automated Teller Machines."
         She waded to the bank and picked up her shoes and stockings.
         When they paused on the bank, she could stand the pain no longer.
         The bank was within reach.
         She found herself walking along the bank, gaze on the souls that were moving.
         I was like one who never casts a look behind, who hesitates before some Rubicon to be crossed, but having touched the farther bank sees no more the shore he has just left."
         Several battalions of soldiers, in their shirt sleeves despite the cold wind, swarme

In [18]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AffinityPropagation
from nltk.stem.snowball import SnowballStemmer

punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = SnowballStemmer("english")

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize)

def get_clusters(sentences):
    tf_idf_matrix = vectorizer.fit_transform(sentences)
    similarity_matrix = (tf_idf_matrix * tf_idf_matrix.T).A
    affinity_propagation = AffinityPropagation(affinity="precomputed", damping=0.5)
    affinity_propagation.fit(similarity_matrix)

    labels = affinity_propagation.labels_
    cluster_centers = affinity_propagation.cluster_centers_indices_

    tagged_sentences = zip(sentences, labels)
    clusters = {}

    for sentence, cluster_id in tagged_sentences:
        clusters.setdefault(sentences[cluster_centers[cluster_id]], []).append(sentence)

    return clusters

sentences = scrape.scrape('bank')


sentences = simpleFilterList(sentences)

clusters = get_clusters(sentences)
print()
for cluster in clusters:
    print(cluster, ':')
    for element in clusters[cluster]:
        print('  - ', element)


A bank cloud building northeast :
  -  A bank cloud building northeast
  -  From airport window watched cloud bank build northwest
  -  A bank cloud tumbled slope left bathing summit cold dampness
  -  In addition storage building owns insurance agency bank bunch commercial real estate
  -  Carmen shook head glancing bank building cloud
  -  We waded gently reverently pulled together smoothly fish thought scared stream feared angler bank came went grandly like cloud float western sky motheropearl flock sometimes form dissolve
Her bank account rarely two hundred :
  -  Her bank account rarely two hundred
  -  Cynthia smiled remembering earlydays confusion opening bed breakfast two newlywed one old man operating piggy bank finance ounce experience among
  -  It thrown across gorge height two hundred fiftyeight foot water supported bank tower solid rock eight hundred foot apart
  -  After breakfast closed account bank turned resignation
  -  I cleaned bank account wrapped withdrawal slip